In [ ]:
"""
Template for week 14 project in Data Visualization

Load binary CT data and plot the contours of the resulting volume
http://graphics.cs.ucdavis.edu/~okreylos/PhDStudies/Spring2000/ECS277/DataSets.html
"""

import struct
import time
import numpy as np
import plotly
import plotly.graph_objects as go
import plotly.figure_factory as ff
import skimage
from skimage import io
import matplotlib.pyplot as plt



In [ ]:
# Note that file names are caps-sensitive on Unix
PLOTS_PATH = "plots/"
DATA_PATH = "data/"
C60_64 = DATA_PATH + "C60_64.vol"
C60_128 = DATA_PATH + "C60_128.vol"
FOOT = DATA_PATH + "Foot.vol"
SKULL = DATA_PATH + "Skull.vol"

VOLS = {}

# Custom colorscale for CT volumes
PL_BONE = [
    [0.0, 'rgb(0, 0, 0)'],
    [0.05, 'rgb(10, 10, 14)'],
    [0.1, 'rgb(21, 21, 30)'],
    [0.15, 'rgb(33, 33, 46)'],
    [0.2, 'rgb(44, 44, 62)'],
    [0.25, 'rgb(56, 55, 77)'],
    [0.3, 'rgb(66, 66, 92)'],
    [0.35, 'rgb(77, 77, 108)'],
    [0.4, 'rgb(89, 92, 121)'],
    [0.45, 'rgb(100, 107, 132)'],
    [0.5, 'rgb(112, 123, 143)'],
    [0.55, 'rgb(122, 137, 154)'],
    [0.6, 'rgb(133, 153, 165)'],
    [0.65, 'rgb(145, 169, 177)'],
    [0.7, 'rgb(156, 184, 188)'],
    [0.75, 'rgb(168, 199, 199)'],
    [0.8, 'rgb(185, 210, 210)'],
    [0.85, 'rgb(203, 221, 221)'],
    [0.9, 'rgb(220, 233, 233)'],
    [0.95, 'rgb(238, 244, 244)'],
    [1.0, 'rgb(255, 255, 255)']
]


In [ ]:
#########################################################################
# Part 1 - Implement Volume class and make_volume()

class Volume:
    """
    Container for CT volume data
    """
    
    def __init__(self, grid_values, grid_extents=(1, 1, 1)):
        """
        Input: 3D numpy array grid_values, optional tuple grid_extents
        that contains extent of grid in each dimension
        
        Ouput: Object storing grid of scalar data
        """
        self._data = grid_values
        self._extents = grid_extents
#         return
    
    # Implement during Part 3
    def plot_volume_contour(self, val, title="Contour plot of volume", plot_edges=True):
        """
        Input: Volume object self, number val, optional string title
        
        Output: plotly figure corresponding to contour plot of volume using Marching Cubes
        with specified value val.  Use grid extents to set aspect ratio.
        Also writes HTML to PLOTS_PATH + title + ".html".
        """

        volume_data = self._data
        verts, faces = skimage.measure.marching_cubes_classic(volume_data, level=val)
        
        fig = ff.create_trisurf(x=verts[:, 0], y=verts[:, 1], z=verts[:, 2], 
                                simplices=faces, plot_edges=plot_edges, title=title)
        
        filepath = './plots/' +title+ ".html"
        plotly.io.write_html(fig, file=filepath)
        fig.show()
        return fig
    
    def frame_args(self, duration):
        """
        frame_args
        """
        return {"frame": {"duration": duration},
                "mode": "immediate",
                "fromcurrent": True,
                "transition": {"duration": duration, "easing": "linear"},}
        
    # Implement during Part 4
    def plot_volume_slice(self, title):
        """
        Input: Volume object self, optional string title
    
        Output: plotly figure corresponding to 3D slices of volume 
        perpendicular to z-axis. The vertical position of this slice 
        should be controlled by buttons and a slider
        Also writes HTML to PLOTS_PATH + title + ".html".
        """   
        vol = self._data
        volume = vol.T

        nb_frames = len(volume)
        
        fig = go.Figure(frames=[go.Frame(data=go.Surface(z=(((len(volume)-1) / 10) - k * 0.1) 
                                                         * np.ones((volume[0].shape[0], 
                                                                    volume[0].shape[1])), 
                                                         surfacecolor=np.flipud(volume[
                                                             len(volume) - 1 - k]), 
                                                         cmin=volume.min(), 
                                                         cmax=volume.max()), 
                                         name=str(k)) for k in reversed(range(nb_frames))])

        # Add data to be displayed before animation starts
        fig.add_trace(go.Surface(z=((len(volume)-1) / 10) * np.ones((volume[0].shape[0], 
                                                                     volume[0].shape[1])), 
                                 surfacecolor=np.flipud(volume[len(volume)-1]), 
                                 colorscale=PL_BONE, cmin=volume.min(), 
                                 cmax=volume.max(), colorbar=dict(thickness=20, ticklen=4)))

        sliders = [{"pad": {"b": 10, "t": 60}, "len": 0.9, "x": 0.1, "y": 0, 
                    "steps": [{"args": [[f.name], self.frame_args(0)], "label": str(k), 
                               "method": "animate",} for k, f in enumerate(fig.frames)],}]

        # Layout
        fig.update_layout(
            title=title,
            width=600,
            height=600,
            scene=dict(zaxis=dict(range=[-0.1, nb_frames/10], autorange=False), 
                       aspectratio=dict(x=self._extents[0], y=self._extents[1], 
                                        z=self._extents[2]), ), 
            updatemenus=[{"buttons": [{"args": [None, self.frame_args(50)], 
                                       "label": "&#9654;", "method": "animate",}, 
                                      {"args": [[None], self.frame_args(0)], 
                                       "label": "&#9724;", "method": "animate",},], 
                          "direction": "left", "pad": {"r": 10, "t": 70}, 
                          "type": "buttons", "x": 0.1, "y": 0,}], sliders=sliders)
        filepath = './plots/' +title+ ".html"
        plotly.io.write_html(fig, file=filepath)
        fig.show()
        
        return fig
        

In [ ]:
def test_volume():
    """ Test the Volume class initializer """
    
    Volume(np.array([[[0]]]))
    Volume(np.array([[[1, 1], [1, 1]], [[2, 2], [2, 2]], [[3, 3], [3, 3]]]), (3, 2, 2))
    
test_volume()

In [ ]:
def make_volume(z_coords, y_coords, x_coords, grid_fun):
    """
    Input: Numpy arrays z_coords, y_coords, x_coords,
    function grid_fun that takes 3 scalar parameters

    Output: Volume object whose grid values as grid_fun(z, y, x)
    taken at points of the grid defined by the coordinate arrays
    """
    z_extent = z_coords[-1] - z_coords[0]
    y_extent = y_coords[-1] - y_coords[0]
    x_extent = x_coords[-1] - x_coords[0]
#     xv, yv, zv = np.meshgrid(x_coords, y_coords, z_coords)
    y_v, z_v, x_v = np.meshgrid(y_coords, z_coords, x_coords)
#     print(xv)
#     print(yv)
#     print(zv)

#     grid_f_v = np.vectorize(grid_fun)
#     grid_f_v(zv, yv, xv)
    grid_fun(z_v, y_v, x_v)
#     print(grid_f_v(zv, yv, xv))

    grid_f = grid_fun(z_v, y_v, x_v)
#     print(grid_fun(zv, yv, xv))

#     return xv, yv, zv
    return Volume(grid_f, grid_extents=(z_extent, y_extent, x_extent))
#     return Volume(np.array([]))

In [ ]:
def test_make_volume():
    """ Test make_volume function """

    z_values = np.linspace(-3, 3, 7)
    y_values = np.linspace(-2, 2, 5)
    x_values = np.linspace(-1, 1, 5)
    grid_fun = lambda z, y, x: x + y + z
    VOLS["linear"] = make_volume(z_values, y_values, x_values, grid_fun)
    
    z_values = np.linspace(-3, 3, 13)
    y_values = np.linspace(-3, 3, 13)
    x_values = np.linspace(-3, 3, 13)
    grid_fun = lambda z, y, x: z ** 2 + y ** 2 + x ** 2
    VOLS["spherical"] = make_volume(z_values, y_values, x_values, grid_fun)
    
test_make_volume()

In [ ]:
############################################################################
# Part 2 - Read binary CT data from a file and create a Volume object


def read_volume(vol_name):
    """
    Input: String vol_name 
    
    Output: Volume object read from binary-encoded CT volume data in given file
    
    NOTE: Use struct module - https://docs.python.org/3/library/struct.html
    """
    with open(vol_name, "rb") as table_file:
        table_binary = table_file.read()
#     print(table_binary)
        
    shape_fmt = ">4i3f"
    table_shape = struct.unpack_from(shape_fmt, table_binary)
    print("Read shape", table_shape)
    
      
    table_size = table_shape[0] * table_shape[1] * table_shape[2]
    table_offset = struct.calcsize(shape_fmt)
    table_fmt = ">" + str(table_size) + "B"
    
    table_bytes = struct.unpack_from(table_fmt, table_binary, table_offset)
    flat_table = np.array(table_bytes, dtype=np.uint8)
    
    vol_shape = np.reshape(flat_table, table_shape[0:3])
    return Volume(vol_shape, grid_extents=(table_shape[4], table_shape[5], table_shape[6]))
#     return np.reshape(flat_table, new_table_shape[0:3])

In [ ]:
def test_read_volume():
    """ Test read_volume() """
    
    VOLS["C60_64"] = read_volume(C60_64)  
    VOLS["FOOT"] = read_volume(FOOT)
    VOLS["SKULL"] = read_volume(SKULL)
    
test_read_volume()

In [ ]:
################################################################################
# Part 3 - Add contour_volume method to the Volume class

def test_plot_volume_contour():
    """ Test plot_volume_contour() """
    
    VOLS["linear"].plot_volume_contour(0, "Contour plot of linear function")    
    VOLS["spherical"].plot_volume_contour(4, "Contour plot of spherical function", plot_edges=False)
    
    VOLS["C60_64"].plot_volume_contour(127, "Contour plot of C60")
    VOLS["FOOT"].plot_volume_contour(127, "Contour plot of human foot")
    VOLS["SKULL"].plot_volume_contour(127, "Contour plot of human skull")
    
#test_plot_volume_contour()

## Links to the 5 triangular meshes produced by `test_plot_volume_contour()`

* [Contour plot of linear function](plots/Contour%20plot%20of%20linear%20function.html)
* [Contour plot of spherical function](plots/Contour%20plot%20of%20spherical%20function.html)
* [Contour plot of C60](plots/Contour%20plot%20of%20C60.html)
* [Contour plot of human foot](plots/Contour%20plot%20of%20human%20foot.html)
* [Contour plot of human skull](plots/Contour%20plot%20of%20human%20skull.html)

In [ ]:
################################################################################
# Part 4 - Add plot_volume_slice method to the Volume class

def test_plot_volume_slice():
    """ Test plot_volume_slice() """

    VOLS["linear"].plot_volume_slice("Horizontal slices of linear function")    
    VOLS["spherical"].plot_volume_slice("Horizontal slices of spherical function")
    
    VOLS["C60_64"].plot_volume_slice("Horizontal slices of C60")
    VOLS["FOOT"].plot_volume_slice("Horizontal slices of human foot")
    VOLS["SKULL"].plot_volume_slice("Horizontal slices of human skull")
    
test_plot_volume_slice()

### Links to the horizontal slicing of the 5 volumes produced by `test_plot_volume_slice()`
* [Horizontal slices of linear function](plots/Horizontal%20slices%20of%20linear%20function.html)
* [Horizontal slices of spherical function](plots/Horizontal%20slices%20of%20spherical%20function.html)
* [Horizontal slices of C60_64](plots/Horizontal%20slices%20of%20C60.html)
* [Horizontal slices of human foot](plots/Horizontal%20slices%20of%20human%20foot.html)
* [Horizontal slices of human skull](plots/Horizontal%20slices%20of%20human%20skull.html)